In [1]:
import pandas as pd
import numpy as np
import sympy
import re

In [175]:
math_train = pd.read_json('number_word_std.dev.json')

In [176]:
math_train.iloc[33:43,:]

,ans,ans_simple,equations,id,index,sources,text
33,11; 13,"[11, 13]","[unkn: 2*k-1, 2*k+1, equ: 3(2*k-1)=7+2*(2*k+1)]",algebra.com.797657,868,algebra.com.797657,find two consecutive odd integers such that th...
34,{11; 3},"[11, 3]","[unkn: x,y, equ: x + y = 14, equ: x^2 + y^2 = ...",yahoo.answers.20061106190512aaqso2e,1381,https://answers.yahoo.com/question/index?qid=2...,The sum of two numbers is fourteen. The sum of...
35,22; 24; 26,"[22, 24, 26]","[unkn: 2*x,2*x+2,2*x+4, equ: 2(2*x+2+2*x+4)-34...",algebra.com.2530,35,algebra.com.2530,Find three consecutive even integers such that...
36,{3; 7},"[3, 7]","[unkn: x,y, equ: x+y=10, equ: 3*y+3=8*x]",algebra.com.2819,36,"[algebra.com.2819, algebra.com.24638, algebra....",The sum of two integers is 10. Three times the...
37,{12; 17},"[12, 17]","[unkn: x,y, equ: x+y=29, equ: 8*x=5*y+11]",algebra.com.457023,610,algebra.com.457023,The sum of two integers is twenty-nine. Eight ...
38,{52; 73},"[52, 73]","[unkn: x,y, equ: x + y = 125, equ: 5*x - 3*y =...",yahoo.answers.20090223121444aar9wxc,1566,https://answers.yahoo.com/question/index?qid=2...,The sum of two numbers is 125. Five times one ...
39,ans_no_result,[],"[unkn: n,n+7, equ: n+(n+7) = 84]",algebra.com.367618,507,algebra.com.367618,find two consecutive multiples of 7 whose sum ...
40,92; 93,"[92, 93]","[unkn: x,x+1, equ: x+(x+1)=185]",algebra.com.449815,606,algebra.com.449815,the sum of two consecutive integers is 185. Fi...
41,90,[90],"[unkn: x, equ: x/3+60=x/15+84]",algebra.com.3318,42,algebra.com.3318,Sixty more than a number divided by three is e...
42,17; 18; 19,"[17, 18, 19]","[unkn: k-1,k,k+1, equ: 2*k=(k+1)+17]",algebra.com.100249,44,algebra.com.100249,Find 3 consecutive numbers such that twice the...


In [177]:
def is_linear(original_string_equation):
    """ Checks if an equation is linear"""
    try:    
        # Strips the 'equ:"
        strip_equ = original_string_equation.lstrip("equ: ")
        # Adds '*' where missing like in 2(k+1) to 2*(k+1)
        reg = re.findall('\d+\(', strip_equ)
        for item in reg:
            reg_with_mul = reg[0][0] + "*" + reg[0][1]
            strip_equ = strip_equ.replace(item, reg_with_mul)
        # Finds the variables
        list_of_var = []
        variables = ""
        for char in strip_equ:
            if char in "abcdefghijklmnopqrstuvwxyxABCDEFGHIJKLMNOPQRSTUVWXYZ":
                if char not in list_of_var:
                    variables = variables + char + " "
                    list_of_var.append(char)
        # Generates an Eq object for separation of the sides of equation
        eq_S = sympy.Eq(*map(sympy.S, strip_equ.split("=")))
        # Combines two sides of the equation
        final_version = eq_S.args[0] + eq_S.args[1]*-1
        # Put it in a list so that the 'linear_eq_to_matrix' can handle
        if final_version.args_cnc()[0] == []:
            final_as_list = final_version.args_cnc()[1]
        else:
            final_as_list = final_version.args_cnc()[0]
        symb = sympy.symbols(variables)
        A, b = sympy.linear_eq_to_matrix(final_as_list, symb)
        if not any(list(A)) and list(b)[0] != 0:
            return False
        else:
            return True
    except:
        return False

In [173]:
trial = [1,'v']
all([type(el) == int for el in trial]) 

False

In [178]:
def is_set_linear(list_of_equations):
    all_linear = True
    list_without_unkn = list_of_equations[1:]
    if not all(map(is_linear, list_without_unkn)):
        return False
    else:
        return True

In [161]:
is_linear(math_train["equations"][0][1])

In [162]:
xy = math_train["equations"][18][1]
xy

'equ: x+y=64'

In [163]:
strip_equ = xy.lstrip("equ: ")
strip_equ

'x+y=64'

In [124]:
reg = re.findall('\d+\(', strip_equ)
for item in reg:
    reg_with_mul = reg[0][0] + "*" + reg[0][1]
    strip_equ = strip_equ.replace(item, reg_with_mul)

In [125]:
list_of_var = []
variables = ""
for char in strip_equ:
    if char in "abcdefghijklmnopqrstuvwxyxABCDEFGHIJKLMNOPQRSTUVWXYZ":
        if char not in list_of_var:
            variables = variables + char + " "
            list_of_var.append(char)
list_of_var 

['x', 'y']

In [126]:
eq_S = sympy.Eq(*map(sympy.S, strip_equ.split("=")))
eq_S

Eq(x + y, 10)

In [127]:
final_version = eq_S.args[0] + eq_S.args[1]*-1
final_version

x + y - 10

In [128]:
if final_version.args_cnc()[0] == []:
    final_as_list = final_version.args_cnc()[1]
else:
    final_as_list = final_version.args_cnc()[0]
final_as_list

[x + y - 10]

In [129]:
symb = sympy.symbols(variables)
symb

(x, y)

In [130]:
A, b = sympy.linear_eq_to_matrix(final_as_list, symb)

In [133]:
listA = list(A)
type(listA[0]) = int

SyntaxError: can't assign to function call (<ipython-input-133-0af3f60f89d2>, line 2)

In [120]:
listA

[y, x]

In [121]:
all([type(el) == Symbol for el in listA])

NameError: name 'Symbol' is not defined

In [105]:
if all([type(el) == int for el in listA]):
    print("y")
else:
    print("n")

n


In [69]:
A

Matrix([[y, x]])

In [70]:
b

Matrix([[20]])

In [179]:
math_train["is_linear"] = math_train["equations"].map(is_set_linear)

In [188]:
math_train[math_train["index"] == 1236]

,ans,ans_simple,equations,id,index,sources,text,is_linear
184,23; 50; 64,"[23, 50, 64]","[unkn: x,y,z, equ: x+y+z=137, equ: y = 4 + 2*x...",algebra.com.18080,1236,algebra.com.18080,The sum of 3 numbers is 137. The second numbe...,True


In [171]:
math_train["is_linear"].value_counts()

True     307
False     67
Name: is_linear, dtype: int64

In [155]:
N_PARAMS = 4 # number of coefficients
def convert_equation_to_vector(original_string_equation):
    """ Converts an equation to a list of parameters of linear combination""" 
    
    # Strips the 'equ:"
    strip_equ = original_string_equation.lstrip("equ: ")
    # Adds '*' where missing like in 2(k+1) to 2*(k+1)
    reg = re.findall('\d+\(', strip_equ)
    for item in reg:
        reg_with_mul = reg[0][0] + "*" + reg[0][1]
        strip_equ = strip_equ.replace(item, reg_with_mul)
    # Finds the variables
    list_of_var = []
    variables = ""
    for char in strip_equ:
        if char in "abcdefghijklmnopqrstuvwxyxABCDEFGHIJKLMNOPQRSTUVWXYZ":
            if char not in list_of_var:
                variables = variables + char + " "
                list_of_var.append(char)
    # Generates an Eq object for separation of the sides of equation
    eq_S = sympy.Eq(*map(sympy.S, strip_equ.split("=")))
    # Combines two sides of the equation
    final_version = eq_S.args[0] + eq_S.args[1]*-1
    # Put it in a list so that the 'linear_eq_to_matrix' can handle
    if final_version.args_cnc()[0] == []:
        final_as_list = final_version.args_cnc()[1]
    else:
        final_as_list = final_version.args_cnc()[0]
    symb = sympy.symbols(variables)
    A, b = sympy.linear_eq_to_matrix(final_as_list, symb)
    negative_b = list(b)[0]
    A_params = list(A)
    params = [*list(A)] #, list(b)[0]*-1]
    # Now vectorizing
    param_A_vector = np.zeros((N_PARAMS,))
    for i, num in enumerate(params):
        param_A_vector[i] = num
    param_vector = np.hstack((param_A_vector, list(b)[0]*-1))
    param_vector_with_closure = np.hstack((8888, param_vector, 9999)) # 8888 - start equation; 9999 - end equation
    
    return param_vector_with_closure

In [200]:
N_EQUATIONS = 3
def form_vectorized_matrix(list_of_unkn_and_equations):
    """Generates a vectoer based on the equations parameters"""
    
    matrix = np.zeros((N_EQUATIONS, N_PARAMS+3)) # Added 3, one for the b in the matrix, and two for the 9999/8888 breakers
    list_of_equtions = list_of_unkn_and_equations[1:]
    for i, equaition in enumerate(list_of_equtions):
        vector = convert_equation_to_vector(equaition)
        matrix[i] = vector
    vectorized_matrix = matrix.reshape((N_PARAMS+3)*N_EQUATIONS,)
    
    return vectorized_matrix

In [187]:
len(list(math_train['equations'][:17].map(form_vectorized_matrix))

17

In [193]:
test_eq = math_train['equations'][184][3]
test_eq

'equ: z = 3*x - 5'

In [192]:
convert_equation_to_vector(test_eq)

array([8888, -3.0, 0.0, 0.0, 0.0, z + 5, 9999], dtype=object)

In [195]:
strip_equ = test_eq.lstrip("equ: ")
strip_equ

'z = 3*x - 5'

In [199]:
form_vectorized_matrix(math_train['equations'][3])

array([  8.88800000e+03,   1.00000000e+00,   1.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -6.80000000e+01,
         9.99900000e+03,   8.88800000e+03,   1.00000000e+00,
        -1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -1.60000000e+01,   9.99900000e+03,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00])